<a href="https://colab.research.google.com/github/umakanetkar/opioid-mortality-corelation-analysis/blob/main/Opioid_Availability_Mortality_Rate_Corelation_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction and Setup

The goal of this project is to see if theres a strong corelation between opioid availabilty and mortality rates in the state of Pennsylvania. There has been a marked rise in drug overdose related deaths and combined with an increase in drug availability, it makes a compelling case to find out this correlation.

To come up with data based conclusions, I am doing analysis on large publically available opioid datasets.


In [62]:
%%capture
!apt install libkrb5-dev
!wget https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install findspark
!pip install sparkmagic
!pip install pyspark
! pip install pyspark --user
! pip install seaborn --user
! pip install plotly --user
! pip install imageio --user
! pip install folium --user
! pip3 install yellowbrick

In [63]:
%%capture
!apt update
!apt install gcc python-dev libkrb5-dev

In [64]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

import os

spark = SparkSession.builder.appName('Opioid-Project').getOrCreate()

In [65]:
!pip install pymongo

In [66]:
import numpy as np
import pandas as pd
import matplotlib

#misc
import gc
import time
import warnings


#graph section
import networkx as nx
#import heapq  # for getting top n number of things from list,dict
import pandas as pd
import numpy as np

# JSON parsing
import json

# HTML parsing
from lxml import etree
import urllib

# SQLite RDBMS
import sqlite3

# Time conversions
import time

# NoSQL DB
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError, OperationFailure

import os
os.environ['SPARK_HOME'] = '/content/spark-2.4.5-bin-hadoop2.7'
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
import pyspark
from pyspark.sql import SQLContext

In [67]:
#spark session
try:
    if(spark == None):
        spark = SparkSession.builder.appName('Initial').getOrCreate()
        sqlContext=SQLContext(spark)
except NameError:
    spark = SparkSession.builder.appName('Initial').getOrCreate()
    sqlContext=SQLContext(spark)

# Opioid Data


I am using data sourced from the [Washington Post](https://www.washingtonpost.com/investigations/interactive/2023/opioid-epidemic-pain-pills-sold-oxycodone-hydrocodone/), in their publication of the DEA's opioid data. I have then selected the PA data from the Washington Post database.

I am chosing to go with this dataset because it was quite recently published (2019) is quite reliable, and contains the features needed for regression analysis. Additionally, data collection is consistent, and metadata is freely available and open-source.

In [ ]:


!wget https://www.washingtonpost.com/wp-stat/dea-pain-pill-database/summary/arcos-pa-statewide-itemized.csv.gz
!gunzip -k arcos-pa-statewide-itemized.csv.gz

--2024-05-28 01:30:30--  https://www.washingtonpost.com/wp-stat/dea-pain-pill-database/summary/arcos-pa-statewide-itemized.csv.gz
Resolving www.washingtonpost.com (www.washingtonpost.com)... 23.208.39.209
Connecting to www.washingtonpost.com (www.washingtonpost.com)|23.208.39.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308798971 (294M) [text/csv]
Saving to: ‘arcos-pa-statewide-itemized.csv.gz.1’

arcos-pa-statewide- 100%[===================>] 294.49M  34.5MB/s    in 9.9s    

2024-05-28 01:30:50 (29.8 MB/s) - ‘arcos-pa-statewide-itemized.csv.gz.1’ saved [308798971/308798971]

gzip: arcos-pa-statewide-itemized.csv already exists; do you wish to overwrite (y or n)? 

Reading the unzipped dataset to spark and taking a look at first few rows to get a general sense of data.

In [ ]:
opioid_sdf = spark.read.csv('arcos-pa-statewide-itemized.csv', header=True)

opioid_sdf.show(5)

Creating a temp view to query this table.

In [ ]:
opioid_sdf.createOrReplaceTempView("opioid_tbl")

From an exploratory data analysis perspective, I am interested to look at the market shares of different quantities (in particular, quantity of units purchased, total MME, and number of transactions) for distributors from year to year.

To do so, we aggregate our data in Spark by grouping by distributor name and transaction year with the following SQL query. This gives the total quantity, MME, and number of transactions for each company from year to year, so it must be divided by the overall yearly total to get the market share; i.e.
market share
t
=
distributor total
t
overall total
t
, where
t
 is the year.

Once the SQL query is complete, I will convert the resulting table to a Pandas dataframe.

In [ ]:
eda_query = '''SELECT BUYER_NAME AS DISTRIBUTOR,
               RIGHT(TRANSACTION_DATE, 4) AS TRANS_YEAR,
               SUM(QUANTITY) AS YEAR_QTY,
               SUM(MME) AS YEAR_MME,
               COUNT(BUYER_NAME) AS YEAR_TRANS
               FROM opioid_tbl
               GROUP BY BUYER_NAME, TRANS_YEAR'''

eda_df = spark.sql(eda_query).toPandas()

In [ ]:
print(str(eda_df.head()) + "\n")
print(str(eda_df.dtypes) + "\n")
print(eda_df.shape)

Description of Columns:


---



DISTRIBUTOR: The pharmacy that sells the opioids directly to consumers.

TRANS_YEAR: The year during which all transactions in the data took place.

YEAR_QTY: The total number of units sold, per the given year and distributor. Units are defined as the number of pills/tablets sold.

YEAR_MME: The total sum of the MME metric, per the given year and distributor. The MME metric is defined as a universal morphine equivalence unit, which determines the strength of each transaction. This takes into account the type of unit, the number of units, the dosage, and type of opioid.

YEAR_TRANS: The total number of individual transactions made, per the given year and distributor.

# Get Market Shares

To convert the yearly quantity, MME, and transaction totals for each distributor to their respective market shares, I am dividing these columns by the overall yearly quantity, MME, and transaction totals.



In [ ]:
eda_df['QTY_MKT_SHARE'] = eda_df.YEAR_QTY
eda_df['MME_MKT_SHARE'] = eda_df.YEAR_MME
eda_df['TRANS_MKT_SHARE'] = eda_df.YEAR_TRANS


def get_qty_mkt_ttl(year):
  return(np.sum(eda_df.YEAR_QTY[eda_df.TRANS_YEAR==year]))

def get_mme_mkt_ttl(year):
  return(np.sum(eda_df.YEAR_MME[eda_df.TRANS_YEAR==year]))

def get_trans_mkt_ttl(year):
  return(np.sum(eda_df.YEAR_TRANS[eda_df.TRANS_YEAR==year]))

years = pd.unique(eda_df['TRANS_YEAR'])

for year in years:
  eda_df.loc[eda_df['TRANS_YEAR']==year, 'QTY_MKT_SHARE'] = \
    eda_df.loc[eda_df['TRANS_YEAR']==year, 'QTY_MKT_SHARE'] / get_qty_mkt_ttl(year)
  eda_df.loc[eda_df['TRANS_YEAR']==year, 'MME_MKT_SHARE'] = \
    eda_df.loc[eda_df['TRANS_YEAR']==year, 'MME_MKT_SHARE'] / get_mme_mkt_ttl(year)
  eda_df.loc[eda_df['TRANS_YEAR']==year, 'TRANS_MKT_SHARE'] = \
    eda_df.loc[eda_df['TRANS_YEAR']==year, 'TRANS_MKT_SHARE'] / get_trans_mkt_ttl(year)

Printing the first 5 rows and first 50 unique distributors.

In [ ]:
print(str(eda_df.head()) + "\n")
print(pd.unique(eda_df.DISTRIBUTOR)[1:50])

I see that we have a lot of instances where we are counting the same distributor multiple number of times. For eg. "'GIANT PHARMACY #6291' 'GIANT PHARMACY #6087' 'GIANT PHARMACY #6112'
 'GIANT PHARMACY #6045'" all correspond to the same "GIANT PHARMACY"

 In further analysis, I will correct this.

# Mortality Data

Moving over to mortality data, I am sourcing it from [CDC Wonder](https://wonder.cdc.gov/cmf-icd10.html). For convenience, I uploaded this dataset to GitHub so that it could be uploaded directly to Colab from the internet.

In [ ]:
# !wget https://github.com/umakanetkar/opioid-mortality-corelation-analysis/blob/main/Compressed-Mortality-1999-2016.txt
!wget https://raw.githubusercontent.com/prmrs2/gthub4455ghdrive/main/Compressed-Mortality-1999-2016.txt

Reading the data into a Pandas data frame.

In [ ]:
mortality_raw = pd.read_csv('Compressed-Mortality-1999-2016.txt', header=0, sep='\t', lineterminator='\r')

mortality_raw.head(675)

Right off the bat, we see the last few columns dont have actual data. We'll take a closer look

In [ ]:
mortality_raw.loc[ 665: ]

As seen above, the rows at the end are just footnotes. This occurs from row 671 onward, so we exclude these rows from the data.

Also worth noting that, each county has an associated row in which the Notes column has the value "Total" and the Year/Year Code columns have NaN values. These rows correspond to the cumulative number of deaths over all years for the county. Since I do not need this information, I will exclude these rows. Also dropping the Notes and County columns, as the information in the Notes column is qualitative and cannot be used for modeling and the information in the County column is encoded by County Code.



In [ ]:
mortality = mortality_raw[0:670]
mortality = mortality.loc[mortality.Notes != '\n"Total"']
mortality = mortality.drop(['Notes', 'County', "Year Code"], axis=1)

Column Descriptions:

County Code: The FIPS code of each county.

Year: The year for the mortality rate for the county.

Deaths: The total number of deaths in the given county and year.

Population: The total population in the given county and year.

Crude Rate: The mortality rate, given as the ratio of number of deaths per 100,000 people in the given county and year.

Because the Crude Rate variable is the mortality rate per 100,000 citizens, I will convert this to the regular death rate (
number of deaths
population size
) to get a value ranging from
[
0
,
1
]
 for later modeling.

In [ ]:
mortality.loc[:, 'Crude Rate'] = mortality.loc[:, 'Crude Rate']/100000

mortality.head(603)

# Zip Code/FIPS Data

One issue is that the mortality data is broken down at the county-level, while the opioid transactions are grouped at the zip code level. In order to aggregate the opioid data to the county-level (uniquely identified by the five digit FIPS code), I obtained a data set from GitHub which relates FIPS codes to zip codes.

In [ ]:
!wget https://raw.githubusercontent.com/Data4Democracy/zip-code-to-county/master/county-fips.csv

Now that data is uploaded into Colab, I will transfer it into Pandas.

In [ ]:
data_type_dict = {'STATEFP': str, 'COUNTYFP': str, 'COUNTYNAME': str}

zips_df = pd.read_csv('county-fips.csv', header=0, \
                      dtype=data_type_dict)[['STATEFP',  'COUNTYFP', \
                                             'COUNTYNAME', 'STATE']]

zips_df['FIPS'] = zips_df.apply(lambda x: float(x[0] + x[1]), axis=1)
zips_df['COUNTYNAME'] = zips_df['COUNTYNAME'].apply(lambda x: x.upper().split(' COUNTY')[0])
zips_df = zips_df[ zips_df['STATE'] == 'PA']

zips_df = zips_df.drop(['STATEFP', 'COUNTYFP', 'STATE'], axis=1)



# Data Cleaning

In the output of the last opioid data cell, different stores from the same company (e.g. different Giant pharmacies) are treated as separate distributors. To remedy this, I am doing some basic text manipulation with regex.

For example, the string 'GIANT PHARMACY #6054' would be converted to 'GIANT PHARMACY' and all trimmed strings are grouped together and columns are aggregated with a sum. So, any distributor that was originally of the form 'GIANT PHARMACY #XXXX' would have their totals and market shares summed together, which is valid as the columns in question are all additive.

In [ ]:
eda_df.head()

In [ ]:
import re

def get_dist_sub(str):
  return(re.split('( [0-9])|( #)', str)[0])

eda_df.loc[:, 'DISTRIBUTOR'] =  eda_df.loc[:, 'DISTRIBUTOR'].apply(get_dist_sub)

eda_df_grouped = eda_df.groupby(by = ['DISTRIBUTOR', 'TRANS_YEAR']).sum().reset_index()

In [ ]:
print(eda_df_grouped.head(30))
# print(pd.unique(eda_df_grouped.DISTRIBUTOR)[1175:1225])

Also assigning, each Distributor a unique index.

In [ ]:
grouped_Dist_list = pd.unique(eda_df_grouped.DISTRIBUTOR)
dict_of_grouped_Dist = {}

for i in range(0, len(grouped_Dist_list)):
  dict_of_grouped_Dist.update({grouped_Dist_list[i] : i})

In [ ]:
# This unique index is then added to eda_df for our later EDA.
eda_df_grouped_Ind = eda_df_grouped.copy()
eda_df_grouped_Ind['Dist_Ind'] = 0

def get_Dist_Ind(dist):
  return(dict_of_grouped_Dist.get(dist))

eda_df_grouped_Ind['Dist_Ind'] = \
  eda_df_grouped_Ind['DISTRIBUTOR'].apply(lambda x: get_Dist_Ind(x))

eda_df_grouped_Ind.head(200)

# EDA

When analyzing providers in Pennsylvania, it's important to consider the major players in the industry. However, using just one metric can be problematic. For instance, a provider might sell a large number of pills, but each pill might have a low opioid concentration. Additionally, some companies might have only recently entered the market. To address this, we need to examine annual trends in market share by quantity, transactions, and MME (morphine milligram equivalent), along with yearly mortality rates. By considering these factors, we can better understand any significant changes in market share.

### Quantity Market Share by Year

I am beginning the EDA with producing a bar plot of the various market shares of annual quantity for different distributors each year.

The most important thing to note is that each year there are only about 5-6 distributors with major proportion of the annual quantity market share. This suggests that in our later modeling, I will likely be able to focus on transactions from the top few companies without losing subtantial information.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

qty_bar_plot = sns.FacetGrid(eda_df_grouped_Ind, col="TRANS_YEAR", \
                             col_wrap=3, sharex=True)
qty_bar_plot.map(plt.plot, "Dist_Ind", "QTY_MKT_SHARE")
qty_bar_plot.set_ylabels('Market Share of Quantity')

In [ ]:
years = eda_df_grouped_Ind.TRANS_YEAR.unique().tolist()

for year in years:
  print(eda_df_grouped_Ind.loc[eda_df_grouped_Ind['TRANS_YEAR']==year, \
      ['DISTRIBUTOR','TRANS_YEAR','QTY_MKT_SHARE','Dist_Ind']].\
      sort_values(by=['QTY_MKT_SHARE'], ascending=False).head(10))
  print("\n")

### MME Market Share by Year

Likewise, we produce a similar bar plot of the market shares of annual MME for different distributors each year.

We again observe that each year's chart is dominated by about 6-7 spikes, further providing credence to only using the top 5 or so distributors when it comes to ML model.

In [ ]:
qty_bar_plot = sns.FacetGrid(eda_df_grouped_Ind, col="TRANS_YEAR", \
                             col_wrap=3, sharex=True)
qty_bar_plot.map(plt.plot, "Dist_Ind", "MME_MKT_SHARE")
qty_bar_plot.set_ylabels('Market Share of MME')


In [ ]:
## Get Top 10 Distributors for each year and their totals:

years = pd.unique(eda_df_grouped_Ind['TRANS_YEAR'])

for year in years:
  print(eda_df_grouped_Ind.loc[eda_df_grouped_Ind['TRANS_YEAR']==year, ['DISTRIBUTOR','TRANS_YEAR','MME_MKT_SHARE','Dist_Ind']].\
             sort_values(by=['MME_MKT_SHARE'], ascending=False).head(10))
  print("\n")


### Transactions Market Share by Year

Also producing an analogous plot for yearly transaction market share.

I see the similar observation holds: each year's plot has several spikes that make up the marjority of the annual transaction total.



In [ ]:
qty_bar_plot = sns.FacetGrid(eda_df_grouped_Ind, col="TRANS_YEAR", \
                             col_wrap=3, sharex=True)
qty_bar_plot.map(plt.plot, "Dist_Ind", "TRANS_MKT_SHARE")
qty_bar_plot.set_ylabels('Market Share of Transactions')

In [ ]:
## Get Top 10 Distributors for each year and their totals:

years = pd.unique(eda_df_grouped_Ind['TRANS_YEAR'])

for year in years:
  print(eda_df_grouped_Ind.loc[eda_df_grouped_Ind['TRANS_YEAR']==year, \
      ['DISTRIBUTOR','TRANS_YEAR','TRANS_MKT_SHARE','Dist_Ind']].\
      sort_values(by=['TRANS_MKT_SHARE'], ascending=False).head(10))
  print("\n")

### Mortality Rate by Year

Finally, producing histogram visualizations to show the distribution of mortality rate per year across counties. We see that these visualizations show roughly normal distributions across years.

In [ ]:
qty_bar_plot = sns.FacetGrid(mortality, col="Year", col_wrap=3, sharex=True)
qty_bar_plot.map(plt.hist, "Crude Rate", bins=15)

# Modelling

Re-querying the opioid data table, this time grouping by zipcode, distributor, and transaction year. Adding the grouping by zipcode so that we can build our model on units uniquely defined by (county, year) like the mortality data.

In [ ]:
opioid_query = '''SELECT
                     BUYER_NAME AS DISTRIBUTOR,
                     BUYER_COUNTY AS COUNTY_NAME,
                     INT(RIGHT(TRANSACTION_DATE, 4)) AS TRANS_YEAR,
                     SUM(QUANTITY) AS YEAR_QTY,
                     SUM(MME) AS YEAR_MME,
                     COUNT(BUYER_NAME) AS YEAR_TRANS
                 FROM opioid_tbl
                 GROUP BY DISTRIBUTOR, COUNTY_NAME, TRANS_YEAR'''

raw_opioid_df = spark.sql(opioid_query).toPandas().dropna()

In [ ]:
raw_opioid_df[['DISTRIBUTOR', 'COUNTY_NAME']]

In [ ]:
#Getting distinct indexes for distributors, as done previously

raw_opioid_df.loc[:, 'DISTRIBUTOR'] =  raw_opioid_df.loc[:, 'DISTRIBUTOR'].apply(get_dist_sub)

opioid_df = raw_opioid_df.groupby(by = ['DISTRIBUTOR', 'TRANS_YEAR', 'COUNTY_NAME']).sum().reset_index()

In [ ]:
print(str(opioid_df.head()) + "\n")
print(str(opioid_df.dtypes) + "\n")
print(str(opioid_df.shape) + "\n")
print(pd.unique(opioid_df.COUNTY_NAME))

###  Calculating Market Shares
Re-calculating the market shares for each distributor for a given year and county.

In [ ]:
opioid_df['QTY_MKT_SHARE'] = opioid_df.YEAR_QTY
opioid_df['MME_MKT_SHARE'] = opioid_df.YEAR_MME
opioid_df['TRANS_MKT_SHARE'] = opioid_df.YEAR_TRANS


def get_qty_mkt_ttl(year, county):
  return(np.sum(opioid_df.loc[(opioid_df.TRANS_YEAR==year) & \
                              (opioid_df.COUNTY_NAME==county), 'YEAR_QTY']))

def get_mme_mkt_ttl(year, county):
  return(np.sum(opioid_df.loc[(opioid_df.TRANS_YEAR==year) & \
                              (opioid_df.COUNTY_NAME==county), 'YEAR_MME']))

def get_trans_mkt_ttl(year, county):
  return(np.sum(opioid_df.loc[(opioid_df.TRANS_YEAR==year) & \
                              (opioid_df.COUNTY_NAME==county), 'YEAR_TRANS']))

years = pd.unique(opioid_df['TRANS_YEAR'])
counties = pd.unique(opioid_df['COUNTY_NAME'])


for year in years:
  for county in counties:
    opioid_df.loc[(opioid_df['TRANS_YEAR']==year) & \
                  (opioid_df['COUNTY_NAME']==county), \
                  'QTY_MKT_SHARE'] = \
      opioid_df.loc[(opioid_df['TRANS_YEAR']==year) & \
                    (opioid_df['COUNTY_NAME']==county), \
                    'QTY_MKT_SHARE'] / get_qty_mkt_ttl(year, county)

    opioid_df.loc[(opioid_df['TRANS_YEAR']==year) & \
                  (opioid_df['COUNTY_NAME']==county), \
                  'MME_MKT_SHARE'] = \
      opioid_df.loc[(opioid_df['TRANS_YEAR']==year) & \
                    (opioid_df['COUNTY_NAME']==county), \
                    'MME_MKT_SHARE'] / get_mme_mkt_ttl(year, county)

    opioid_df.loc[(opioid_df['TRANS_YEAR']==year) & \
                  (opioid_df['COUNTY_NAME']==county), \
                  'TRANS_MKT_SHARE'] = \
      opioid_df.loc[(opioid_df['TRANS_YEAR']==year) & \
                    (opioid_df['COUNTY_NAME']==county), \
                    'TRANS_MKT_SHARE'] / get_trans_mkt_ttl(year, county)

In [ ]:
opioid_df.head(10)

### Join Opioid and FIPS Data

Combining the opioid data with the FIPS dataset to aggregate to the county-level, in order to match up with our mortality data.

In [ ]:
merged_opioid_df = opioid_df.merge(zips_df, \
                                   left_on='COUNTY_NAME', \
                                   right_on='COUNTYNAME').\
                                   drop_duplicates().\
                                   drop(['COUNTY_NAME', 'COUNTYNAME'], axis=1)

print(str(merged_opioid_df.head(123892)) + "\n")
print(str(merged_opioid_df.shape) + "\n")


merged_opioid_df

### Aggregate to County-Level

Aggregating data to the county-level, using FIPS in the groupby.

In [ ]:
final_opioid_df = merged_opioid_df.groupby(by = ['DISTRIBUTOR', 'TRANS_YEAR', 'FIPS']).sum().reset_index()

final_opioid_df

In [ ]:
final_opioid_df.head()

### Reformatting Opioid Data

Now that opioid data is prepped, I will reformate it so that it can be merged with the mortality data. This is because the mortality data rows are uniquely identified by (county, year), while the opioid data rows are uniquely identified by (county, year, distributor).

I am selecting the top 5 distributors for each type of market share for every (county, year) pair. These are used as columns in a new data frame that's uniquely identified by (county, year) pairs. For any (county, year) pair that doesn't have 5 distributors,I will simply append NaN values.

In [ ]:
new_opioid_df = final_opioid_df.copy(deep=True)

fips = pd.unique(new_opioid_df['FIPS'])
years = pd.unique(new_opioid_df['TRANS_YEAR'])

frames = []

for fip in fips:
  for year in years:
    df_QTY = new_opioid_df.loc[(new_opioid_df['FIPS']==fip) & \
                               (new_opioid_df['TRANS_YEAR']==year), \
        ['TRANS_YEAR', 'FIPS', 'QTY_MKT_SHARE', 'MME_MKT_SHARE', 'TRANS_MKT_SHARE']].\
        sort_values(by=['QTY_MKT_SHARE'], ascending=False).head(5)
    df_MME = new_opioid_df.loc[(new_opioid_df['FIPS']==fip) & \
                               (new_opioid_df['TRANS_YEAR']==year), \
        ['TRANS_YEAR', 'FIPS', 'QTY_MKT_SHARE', 'MME_MKT_SHARE', 'TRANS_MKT_SHARE']].\
        sort_values(by=['MME_MKT_SHARE'], ascending=False).head(5)
    df_TRANS = new_opioid_df.loc[(new_opioid_df['FIPS']==fip) & \
                                 (new_opioid_df['TRANS_YEAR']==year), \
        ['TRANS_YEAR', 'FIPS', 'QTY_MKT_SHARE', 'MME_MKT_SHARE', 'TRANS_MKT_SHARE']].\
        sort_values(by=['TRANS_MKT_SHARE'], ascending=False).head(5)

    vals = list()
    vals.append(year)
    vals.append(fip)

    # If there are at least 5 distributors for this (FIPS, year) pair, take all
    # of them
    if(len(df_QTY) >= 5):
      vals.append(df_QTY.iloc[0]['QTY_MKT_SHARE'])
      vals.append(df_MME.iloc[0]['MME_MKT_SHARE'])
      vals.append(df_TRANS.iloc[0]['TRANS_MKT_SHARE'])

      vals.append(df_QTY.iloc[1]['QTY_MKT_SHARE'])
      vals.append(df_MME.iloc[1]['MME_MKT_SHARE'])
      vals.append(df_TRANS.iloc[1]['TRANS_MKT_SHARE'])

      vals.append(df_QTY.iloc[2]['QTY_MKT_SHARE'])
      vals.append(df_MME.iloc[2]['MME_MKT_SHARE'])
      vals.append(df_TRANS.iloc[2]['TRANS_MKT_SHARE'])

      vals.append(df_QTY.iloc[3]['QTY_MKT_SHARE'])
      vals.append(df_MME.iloc[3]['MME_MKT_SHARE'])
      vals.append(df_TRANS.iloc[3]['TRANS_MKT_SHARE'])

      vals.append(df_QTY.iloc[4]['QTY_MKT_SHARE'])
      vals.append(df_MME.iloc[4]['MME_MKT_SHARE'])
      vals.append(df_TRANS.iloc[4]['TRANS_MKT_SHARE'])

    # Otherwise, take as many as there are and append NaN for the rest
    else:
      ind = 0

      for i in range(len(df_QTY)):
        vals.append(df_QTY.iloc[i]['QTY_MKT_SHARE'])
        vals.append(df_QTY.iloc[i]['MME_MKT_SHARE'])
        vals.append(df_QTY.iloc[i]['TRANS_MKT_SHARE'])
        ind = ind+1
      while(ind < 5):
        vals.append(np.nan)
        vals.append(np.nan)
        vals.append(np.nan)
        ind = ind+1

    df_per_year = pd.DataFrame([vals],
        columns= ['TRANS_YEAR', 'FIPS', 'QTY_MKT_SHARE_1', 'MME_MKT_SHARE_1', 'TRANS_MKT_SHARE_1', \
                  'QTY_MKT_SHARE_2', 'MME_MKT_SHARE_2', 'TRANS_MKT_SHARE_2', \
                  'QTY_MKT_SHARE_3', 'MME_MKT_SHARE_3', 'TRANS_MKT_SHARE_3', \
                  'QTY_MKT_SHARE_4', 'MME_MKT_SHARE_4', 'TRANS_MKT_SHARE_4', \
                  'QTY_MKT_SHARE_5', 'MME_MKT_SHARE_5', 'TRANS_MKT_SHARE_5', \
                  ])
    frames.append(df_per_year)

result_df = pd.concat(frames).reset_index(drop=True)

result_df #67 unique fips, 9 unique years = 603 rows


### Merge Opioid and Mortality Data

Because the opioid data is finally in correct format, I am merging it with the mortality data to get the final data frame for the modeling.

In [ ]:
model_df = mortality.merge(result_df, \
                           left_on=['County Code', 'Year'], \
                           right_on = ['FIPS', 'TRANS_YEAR']).\
                           drop(['County Code', 'Deaths', \
                                 'Population', 'TRANS_YEAR'], axis=1)

model_df = model_df.loc[:, ['FIPS', 'Year', 'Crude Rate', \
                           'QTY_MKT_SHARE_1', 'QTY_MKT_SHARE_2', \
                           'QTY_MKT_SHARE_3', 'QTY_MKT_SHARE_4', \
                           'QTY_MKT_SHARE_5', \
                           'MME_MKT_SHARE_1', 'MME_MKT_SHARE_2', \
                           'MME_MKT_SHARE_3', 'MME_MKT_SHARE_4', \
                           'MME_MKT_SHARE_5', \
                           'TRANS_MKT_SHARE_1', 'TRANS_MKT_SHARE_2', \
                           'TRANS_MKT_SHARE_3', 'TRANS_MKT_SHARE_4', \
                           'TRANS_MKT_SHARE_5']]

model_df

In [ ]:
#Dropping NaNs
model_df_drop = model_df.dropna()
model_df_drop

Splitting the independent and dependent variables into separate objects.

In [ ]:
label = model_df_drop['Crude Rate']
features = model_df_drop.loc[:, model_df_drop.columns != 'Crude Rate']

### Test Train Split

Now splitting the data using a 30/70 testing/training split for validation. Fixing the random seed for the split so that the analysis is consistent across notebook runs.

### Linear Regression

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = \
  train_test_split(features, label, test_size=0.3, random_state=11)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lm = LinearRegression()
lm_model = lm.fit(x_train, y_train)
lm_y_pred = lm.predict(x_test)

In [ ]:
print('MSE: ' + str(mean_squared_error(y_test, lm_y_pred)))

In [ ]:
lm_score = lm.score(x_test, y_test)
print('R^2: ' + str(lm_score))

Although an
R
2
 value of ~0.19 may not seem particularly high, it is important to note that this means that just by looking at the market shares top 5 distributors for each (county, year) pair, we account for almost 20% of the variability in the annual mortality rate for each county. The strength of the association between these market shares and the mortality rate corroborates the notion that a leading cause of death in PA prior to the COVID-19 pandemic were deaths related opioid-usage.

Now that we have looked at the overall model performance, we dig down to look at the specific model coefficient values.

In [ ]:
print(lm.coef_)

### Lasso Regression

Now trying a LASSO regression model to see if the regularization/shrinkage effect that this model has on the coefficients improves the performance.

To determine the value of the
α
 parameter, I am using 1000 rounds of 5 fold cross-validation with a fixed random seed for consistent runs of the notebook.

In [ ]:
from sklearn.linear_model import LassoCV

lasso = LassoCV(eps=0.0001, n_alphas=1000, cv=5, random_state=19)
lasso_model = lasso.fit(x_train, y_train)
lasso_y_pred = lasso.predict(x_test)

Once again, checking the MSE and
R
2
 to compare performances.

In [ ]:
print('MSE: ' + str(mean_squared_error(y_test, lasso_y_pred)))

In [ ]:
lasso_score = lasso.score(x_test, y_test)
print('R^2: ' + str(lasso_score))

V slight increase in MSE and decrease in R2 as compared to linear model.
Checking the co-efficients to see if any coefficient was shrunk to 0.

In [ ]:
print(lasso_model.coef_)

### Ridge Regression

Now trying a ridge regression model which penalizes large coefficients, but not to the same extent as a LASSO regression.

To determine the value of
α
,  using 1000 rounds of 5-fold cross validation.

In [ ]:
from sklearn.linear_model import RidgeCV

ridge = RidgeCV(alphas=[n/10000 for n in range(1, 1000)], cv=5)
ridge_model = ridge.fit(x_train, y_train)
ridge_y_pred = ridge.predict(x_test)

Looking at the model performance metrics to determine the effectiveness of the ridge regression.

In [ ]:
print('MSE: ' + str(mean_squared_error(y_test, ridge_y_pred)))

In [ ]:
ridge_score = ridge.score(x_test, y_test)
print('R^2: ' + str(ridge_score))

As compared to previous LASSO model, we see theres a slight uptick in R2 and decrease in MSE. This seems to suggest that indeed all of the chosen covariates can improve prediction of the mortality rates.

# Conclusion and Discussion


 In this project, I modeled annual county mortality rate as a function of the market shares of opioid transactions, quantity of opioid units sold to distributors, and MME of the top 5 major distributors in each county every year.

Of all the models produced, the OLS (linear regression) seemed to perform the best, followed by the LASSO model, and finally the ridge model. The OLS model outperformed the other two models in terms of both MSE and
R
2
. The fact that the other two models both implement some form of shrinkage of the covariate coefficients and performed worse indiciates that these variables are useful in explaining some of the variability in the annual county-level mortality in PA counties from 2006-2014.

Although an
R
2
 of 0.19 may seem unassuming at a first glance, it is important to note that we are modeling all annual county-level mortality as a function of these opioid-related market shares. The fact that these covariates can explain almost 20% of this variability is further potential evidence of the fact that there is a strong relationship between opioid activity and mortality in PA, possibly even being an active cause of mortality.

One of the biggest issues I faced during this project was getting various data sources together. For instance, the mortality data was identified by year and FIPS code, while the opioid data was identified by year and county name. Because of issues like this, I had to get external data sets to get columns to match up for our joins. It was also initially difficult finding data sources located online to make this notebook functional entirely in the cloud, but I got around this by using GitHub.